### Install library

In [1]:
# !pip3 install ktrain --user

  Using cached https://files.pythonhosted.org/packages/3a/83/e74092e7f24a08d751aa59b37a9fc572b2e4af3918cb66f7766c3affb1b4/transformers-3.5.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/80/72/a4fba7559978de00cf44081c548c5d294bf00ac7dcda2db405d2baa8c67a/cchardet-2.1.7-cp37-cp37m-manylinux2010_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/9b/cd/dc52755d30ba41c60243235460961fc28022e5b6731f16c268667625baea/networkx-2.5-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/bf/4c/cb7da76f3a5e077e545f9cf8575b8f488a4e8ad60490838f89c5cdd5bb57/pandas-1.1.4-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/ba/19/24d0f1f454a2c1eb689ca28d2f178db81e5024f42d82729a4ff6771155cf/Whoosh-2.7.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f4/cb/64623369f348e9bfb29ff898a57ac7c91ed4921f228e9726546614d63ccb/scikit_learn-0.23.2-cp37-cp37m-manylinux1_x86_64.whl
  Using cached htt

  The script sacremoses is installed in '/home/basharm/.local/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  The script transformers-cli is installed in '/home/basharm/.local/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Found existing installation: Pillow 6.0.0
    Uninstalling Pillow-6.0.0:
      Successfully uninstalled Pillow-6.0.0
  The script bokeh is installed in '/home/basharm/.local/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Found existing installation: scikit-learn 0.19.2
    Uninstalling scikit-learn-0.19.2:
      Successfully uninstalled scikit-learn-0.19.2


### Import Libraries

In [6]:
# General libraries
import pandas as pd
import numpy as np

# import speficic libraries
import tensorflow as tf
import ktrain # Ktrain is a "Lightweight Wrapper" for Keras to Help Train Neural Networks
from ktrain import text

# custom libraries
from simple_clean import CleanTweets
from evaluate_classification import EvaluateBinaryClassification

In [7]:
# Declare Reload Magic Function
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [8]:
# Initialise Random Variables
SEED = 123
np.random.seed(SEED)

### Load Data

In [3]:
df_train = pd.read_csv('LearningLossModelQMI_5.csv')
df_train.head(5)

,text,label
0,my dad is a whore we were supposed to leave to...,0
1,xatp he did you a favor then . don ’ t need no...,1
2,xatp kill yourself you disgusting whore of a d...,1
3,xatp xatp do you feel hurt by slut shaming ! s...,0
4,"`` i have those shoes ! take my shoes off , wh...",0


In [4]:
df_test = pd.read_csv('QmiTest.csv')
df_test.head(5)

,label,text
0,1,@ winegirl73 2 bit attention seeking whore
1,0, i was dodging bullets in slow motion you slut 
2,0,@ mayoroflondon sadiq khan-proud mayor of the ...
3,0,"@ lewdpon3 i agree , you should definitely lea..."
4,1,@ peoples_pundit @ fluffydogattack @ sengillib...


In [9]:
ct = CleanTweets()
df_train['text'] = df_train['text'].apply(ct.prepro_a_tweet)
df_train.head(5)

,text,label
0,my dad is a whore we were supposed to leave to...,0
1,xatp he did you a favor then . don ’ t need no...,1
2,xatp kill yourself you disgusting whore of a d...,1
3,xatp xatp do you feel hurt by slut shaming ! s...,0
4,"`` i have those shoes ! take my shoes off , wh...",0


In [11]:
df_test['text'] = df_test['text'].apply(ct.prepro_a_tweet)
df_test.head(5)

,label,text
0,1,xatp 2 bit attention seeking whore
1,0, i was dodging bullets in slow motion you slut 
2,0,xatp sadiq khan-proud mayor of the rape capita...
3,0,"xatp i agree , you should definitely leave the..."
4,1,xatp xatp xatp xatp did she deny being a slut ?


### Prepare data in model format

In [12]:
TEXT_FIELD = 'text'
LABEL_FIELD = 'label'

In [13]:
X_train, y_train = df_train[TEXT_FIELD].to_list(), df_train[LABEL_FIELD].to_list()
X_test, y_test = df_test[TEXT_FIELD].to_list(), df_test[LABEL_FIELD].to_list()

In [14]:
(xtrain,  ytrain), (xtest, ytest), preproc = text.texts_from_array(x_train=X_train, y_train=y_train,
                                                                       x_test=X_test, y_test=y_test,
                                                                       class_names=[0, 1],
                                                                       preprocess_mode='bert',
                                                                       maxlen=350, 
                                                                       max_features=35000)

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


task: text classification


In [15]:
model = text.text_classifier(name='bert', train_data=(xtrain, ytrain), preproc=preproc)

Is Multi-Label? False
maxlen is 350
done.


In [16]:
learner = ktrain.get_learner(model=model,
                             train_data = (xtrain, ytrain),
                             val_data = (xtest, ytest),
                             batch_size = 6)

In [17]:
learner.fit_onecycle(lr=2e-5, epochs=1)



begin training using onecycle policy with max lr of 2e-05...
500/500 [==============================] - 1937s 4s/step - loss: 0.4627 - accuracy: 0.7813 - val_loss: 0.3503 - val_accuracy: 0.8544


### Creat a predictor to make prediction on unknown data

In [18]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [19]:
predictor.get_classes()

[0, 1]

In [20]:
predicted = predictor.predict(X_test)

In [23]:
ebc = EvaluateBinaryClassification(gnd_truths = y_test, predictions = predicted)
print(ebc.get_full_report())

EvaluateBinaryClassification Object Created

Total Samples	1003
Positive Samples	362
Negative Samples	641
True Positive	306
True Negative	551
False Positive	90
False Negative	56
Accuracy	0.8544366899302094
Precision	0.7727272727272727
Recall	0.8453038674033149
F1 Measure	0.8073878627968337
Cohen Kappa Score	0.6907785543847794
Area Under Curve	0.852449125589333

              precision    recall  f1-score   support

           0       0.91      0.86      0.88       641
           1       0.77      0.85      0.81       362

    accuracy                           0.85      1003
   macro avg       0.84      0.85      0.85      1003
weighted avg       0.86      0.85      0.86      1003



### We can store our trained model

In [24]:
# let's save the predictor for later use
predictor.save('/tmp/my_predictor')

In [25]:
# reload the predictor
reloaded_predictor = ktrain.load_predictor('/tmp/my_predictor')

In [26]:
# make a prediction on the same document to verify it still works
reloaded_predictor.predict(X_test[0:10])

[1, 0, 0, 0, 1, 0, 0, 0, 1, 0]